In [ ]:
import numpy as np
import pandas as pd

filename = '../npz/BTCUSD_1H.npz'
data = np.load(filename, allow_pickle=True)['data']
# преобразование в формат pandas
df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

# Преобразование 'timestamp' в datetime и установка его как индекс DataFrame
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

# Преобразование всех столбцов с ценами и объемом в числовой формат
for column in ['open', 'high', 'low', 'close', 'volume']:
    df[column] = pd.to_numeric(df[column], errors='coerce')

In [ ]:
from flask import Flask, render_template, jsonify, request
import threading
import os
import numpy as np

# Получаем абсолютный путь к текущей директории, в которой находится Jupyter Notebook
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
static_folder = os.path.join(notebook_dir, 'static')
template_folder = os.path.join(notebook_dir, 'templates')

app = Flask(__name__, static_folder=static_folder, template_folder=template_folder)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/2d.html')
def two_d():
    return render_template('2d.html')

@app.route('/ohlc-data', methods=['GET'])
def get_ohlc_data():
    len = 1440
    # dfsl = df[:len]
    dfsl = df
    return jsonify({ 'close': dfsl['close'].to_list(), 'vol': df['volume'].to_list() })

@app.route('/server_info')
def server_info():
    server_software = request.environ.get('SERVER_SOFTWARE', '')
    return f"Server Software: {server_software}"

# Функция для запуска Flask в потоке
def run_flask():
    app.run(port=8050, debug=True, use_reloader=False)

# Запуск Flask в отдельном потоке
flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

In [ ]:
from IPython.display import IFrame

# Встраиваем страницу Flask в Jupyter Notebook
IFrame(src="http://localhost:8050", width=660, height=500)
